# TP2 Apprentissage semi-supervisé
### ANTOINE Maxime, DUCRUET Thibault

Imports

In [110]:
import numpy as np
np.set_printoptions(threshold=10000,suppress=True)
import pandas as pd
from scipy.stats import mode
import warnings

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

warnings.filterwarnings('ignore')

## Données
Charger les données

In [111]:
# Chargement du fichier
def chargement_fichier(f):
    # Lecture du fichier avec pandas du dossier data
    data = pd.read_csv("data/" + f, sep=r"\s+", engine="python", header=None)

    # Séparation des variables
    X = data.iloc[:, :-1].astype(float)  # récupère les  colonnes de données
    y = data.iloc[:, -1]                 # dernière colonne (cible)

    print("X shape :", X.shape)
    print("y shape :", y.shape)
    return X, y


Découpage des données en apprentissage et test

In [112]:
def decoupage_donnees(X, y):
    # Découpage stratifié
    X_A, X_T, y_A, y_T = train_test_split(
        X, y,
        test_size=1/3,
        stratify=y,
        random_state=42
    )

    print("Taille apprentissage :", X_A.shape)
    print("Taille test :", X_T.shape)
    return X_A, X_T, y_A, y_T


Affichage des metrics de résultats de test

In [113]:
def display_metrics(y_T_enc, y_pred_mlp_norm):    
    accuracy_mlp = accuracy_score(y_T_enc, y_pred_mlp_norm)
    print("Accuracy MLP :", accuracy_mlp)

    conf_matrix_mlp = confusion_matrix(y_T_enc, y_pred_mlp_norm)
    print("Matrice de confusion MLP :\n", conf_matrix_mlp)

    # Précision pour chaque classe
    precision = precision_score(y_T_enc, y_pred_mlp_norm, average=None, zero_division=0)
    print("\nPrécision par classe:")
    for i, p in enumerate(precision):
        print(f"  Classe {i+1}: {p:.4f}")

    # Rappel pour chaque classe
    recall = recall_score(y_T_enc, y_pred_mlp_norm, average=None, zero_division=0)
    print("\nRappel par classe:")
    for i, r in enumerate(recall):
        print(f"  Classe {i+1}: {r:.4f}")

### PERCEPTRON MULTI CLASSE

Classe de perceptron multi classe

In [114]:
class PerceptronMulticlasse:
    def __init__(self, n_classes, n_features, lr=0.1, n_iter=100):
        self.n_classes = n_classes
        self.n_features = n_features
        self.lr = lr
        self.n_iter = n_iter
        self.W = np.zeros((n_classes, n_features))

    def fit(self, X, y):
        for _ in range(self.n_iter):
            for xk, yk in zip(X, y):

                # scores pour chaque classe
                scores = np.dot(self.W, xk)   # Produit scalaire

                j = np.argmax(scores)         # classe prédite
                i = yk                        # classe réelle

                if i != j:
                    self.W[i] += self.lr * xk
                    self.W[j] -= self.lr * xk

    def predict(self, X):
        scores = np.dot(X, self.W.T)          # (n_samples, n_classes)
        return np.argmax(scores, axis=1)


Sans normalisation

In [115]:
from sklearn.preprocessing import LabelEncoder

def evaluation_perceptron_multiclasse(X_A, X_T, y_A, y_T):

    # Encodage des classes
    encoder = LabelEncoder()
    y_A_enc = encoder.fit_transform(y_A)
    y_T_enc = encoder.transform(y_T)

    X_A = np.asarray(X_A)
    X_T = np.asarray(X_T)

    # Initialisation et entraînement du perceptron multiclasse
    n_classes = len(np.unique(y_A_enc))
    n_features = X_A.shape[1]

    perceptron = PerceptronMulticlasse(
        n_classes=n_classes,
        n_features=n_features,
        lr=0.1,
        n_iter=100
    )
    
    perceptron.fit(X_A, y_A_enc)

    y_pred = perceptron.predict(X_T)

    print("=== PERCEPTRON MULTICLASSE ===")
    display_metrics(y_T_enc, y_pred)


Avec normalisation

In [116]:
def evaluation_perceptron_multiclasse_normalise(X_A, X_T, y_A, y_T):

    encoder = LabelEncoder()

    y_A_enc = encoder.fit_transform(y_A)
    y_T_enc = encoder.transform(y_T)
    scaler = StandardScaler()

    # Apprentissage de la normalisation sur A
    X_A_norm = scaler.fit_transform(X_A)

    # Application de la même normalisation sur T
    X_T_norm = scaler.transform(X_T)

   # Initialisation et apprentissage
    n_classes = len(np.unique(y_A_enc))
    n_features = X_A_norm.shape[1]

    perceptron = PerceptronMulticlasse(
        n_classes=n_classes,
        n_features=n_features,
        lr=0.1,
        n_iter=100
    )

    perceptron.fit(X_A_norm, y_A_enc)

    # Prédictions
    y_pred_perceptron_norm = perceptron.predict(X_T_norm)

    print("=== PERCEPTRON MULTICLASSE AVEC NORMALISATION ===")

    display_metrics(y_T_enc, y_pred_perceptron_norm)

### PERCEPTRON MULTI COUCHE
Sans normalisation

In [117]:
def evaluation_mlp(X_A, X_T, y_A, y_T):
    encoder = LabelEncoder()

    y_A_enc = encoder.fit_transform(y_A)
    y_T_enc = encoder.transform(y_T)

    mlp = MLPClassifier(
        hidden_layer_sizes=(3,),
        activation='relu',
        solver='adam',
        max_iter=1000,
        random_state=10
    )

    mlp.fit(X_A, y_A_enc)
    y_pred_mlp = mlp.predict(X_T)

    print("=== MLP SANS NORMALISATION ===")

    display_metrics(y_T_enc, y_pred_mlp)

Avec normalisation

In [118]:
def evaluation_mlp_normalise(X_A, X_T, y_A, y_T):
    encoder = LabelEncoder()

    y_A_enc = encoder.fit_transform(y_A)
    y_T_enc = encoder.transform(y_T)
    scaler = StandardScaler()

    # Apprentissage de la normalisation sur A
    X_A_norm = scaler.fit_transform(X_A)

    # Application de la même normalisation sur T
    X_T_norm = scaler.transform(X_T)

    mlp_norm = MLPClassifier(
        hidden_layer_sizes=(3,),
        activation='relu',
        solver='adam',
        max_iter=1000,
        random_state=42
    )

    mlp_norm.fit(X_A_norm, y_A_enc)

    y_pred_mlp_norm = mlp_norm.predict(X_T_norm)

    print("=== MLP AVEC NORMALISATION ===")

    display_metrics(y_T_enc, y_pred_mlp_norm)


### Evaluation des performances

In [119]:
def traitement(f):
    X, y = chargement_fichier(f)
    X_A, X_T, y_A, y_T = decoupage_donnees(X, y)
    evaluation_perceptron_multiclasse(X_A, X_T, y_A, y_T)
    evaluation_perceptron_multiclasse_normalise(X_A, X_T, y_A, y_T)
    evaluation_mlp(X_A, X_T, y_A, y_T)
    evaluation_mlp_normalise(X_A, X_T, y_A, y_T)

#Les fichiers doivent être présent dans un dossier data
datasources = ["iris", "glass", "Lsun", "Wave", "breast-cancer-wisconsin"]
for ds in datasources :
    print("-------------")
    print(ds)
    print("-------------")
    traitement(ds + ".txt")

-------------
iris
-------------
X shape : (150, 4)
y shape : (150,)
Taille apprentissage : (100, 4)
Taille test : (50, 4)
=== PERCEPTRON MULTICLASSE ===
Accuracy MLP : 0.86
Matrice de confusion MLP :
 [[15  1  0]
 [ 0 17  0]
 [ 0  6 11]]

Précision par classe:
  Classe 1: 1.0000
  Classe 2: 0.7083
  Classe 3: 1.0000

Rappel par classe:
  Classe 1: 0.9375
  Classe 2: 1.0000
  Classe 3: 0.6471
=== PERCEPTRON MULTICLASSE AVEC NORMALISATION ===
Accuracy MLP : 0.74
Matrice de confusion MLP :
 [[16  0  0]
 [ 0 11  6]
 [ 0  7 10]]

Précision par classe:
  Classe 1: 1.0000
  Classe 2: 0.6111
  Classe 3: 0.6250

Rappel par classe:
  Classe 1: 1.0000
  Classe 2: 0.6471
  Classe 3: 0.5882
=== MLP SANS NORMALISATION ===
Accuracy MLP : 1.0
Matrice de confusion MLP :
 [[16  0  0]
 [ 0 17  0]
 [ 0  0 17]]

Précision par classe:
  Classe 1: 1.0000
  Classe 2: 1.0000
  Classe 3: 1.0000

Rappel par classe:
  Classe 1: 1.0000
  Classe 2: 1.0000
  Classe 3: 1.0000
=== MLP AVEC NORMALISATION ===
Accuracy 

### BAGGING
Echantillonage des données d'apprentissage

In [120]:
def bootstrap_sample(X, y):
    n = X.shape[0]
    # Tirage avec remise
    indices = np.random.choice(n, size=n, replace=True)
    return X[indices], y[indices]


Apprentissage avec Bagging avec K MLP

In [121]:
def train_bagging_mlp(X, y, K):
    models = []

    for k in range(K):
        X_boot, y_boot = bootstrap_sample(X, y)

        mlp = MLPClassifier(
            hidden_layer_sizes=(3,),
            activation='relu',
            solver='adam',
            max_iter=1000,
            random_state=k
        )

        mlp.fit(X_boot, y_boot)
        models.append(mlp)

    return models


Prédiction avec Bagging

In [122]:
def bagging_predict(models, X):
    # Prédictions de tous les modèles
    all_preds = np.array([model.predict(X) for model in models])

    # Vote majoritaire
    y_pred, _ = mode(all_preds, axis=0)
    return y_pred.ravel()

Evaluation avec Bagging

In [123]:
def evaluation_bagging_mlp(X_A, X_T, y_A, y_T, K):
    encoder = LabelEncoder()

    y_A_enc = encoder.fit_transform(y_A)
    y_T_enc = encoder.transform(y_T)

    X_A = np.asarray(X_A)
    X_T = np.asarray(X_T)
    y_A_enc = np.asarray(y_A_enc)
    y_T_enc = np.asarray(y_T_enc)

    # Entraînement du bagging
    models = train_bagging_mlp(X_A, y_A_enc, K)

    # Prédictions avec vote majoritaire
    y_pred_bagging = bagging_predict(models, X_T)

    # Évaluation
    print("=== BAGGING DE MLP ===")

    display_metrics(y_T_enc, y_pred_bagging)

Code de traitement pour Bagging

In [124]:
def traitement_bagging(f, K=10):
    X, y = chargement_fichier(f)
    X_A, X_T, y_A, y_T = decoupage_donnees(X, y)
    evaluation_bagging_mlp(X_A, X_T, y_A, y_T, K)

traitement_bagging("iris.txt", K=10)

X shape : (150, 4)
y shape : (150,)
Taille apprentissage : (100, 4)
Taille test : (50, 4)
=== BAGGING DE MLP ===
Accuracy MLP : 1.0
Matrice de confusion MLP :
 [[16  0  0]
 [ 0 17  0]
 [ 0  0 17]]

Précision par classe:
  Classe 1: 1.0000
  Classe 2: 1.0000
  Classe 3: 1.0000

Rappel par classe:
  Classe 1: 1.0000
  Classe 2: 1.0000
  Classe 3: 1.0000
